In [4]:
import os

import numpy as np
import pandas as pd
from base import Experiment

In [5]:
%matplotlib qt

In [6]:
# Collect data
M_but = 74.12
M_ocm = 418.4
M_peta = 298.29
M_dmag = 198.12

but_rho = 0.82668

PATH = "D:\WORKS\Diplom-work\Experiments\Viscosity"


def to_df(exp: Experiment):
    df = pd.DataFrame()
    k = 1
    df = exp.d.copy(deep=True).rename(columns={"x": "Temperature", "y": "D"})
    df["Viscosity"] = k * (df["Temperature"] + 273.15) / (df["D"] * 0.001)

    df["compound"] = exp.info["compound"]
    df["rho"] = exp.info["rho"]
    df["w_mass"] = exp.info["w"]
    df["w_vol"] = df["w_mass"] * df["rho"] / but_rho
    df["D0"] = exp.info["D0"]
    df["E"] = exp.info["E"]
    df['Kin_viscosity'] = df['Viscosity']/df["rho"]
    df = df[
        [
            "compound",
            "rho",
            "w_mass",
            "w_vol",
            "Temperature",
            "time",
            "Viscosity",
            'Kin_viscosity',
            "D",
            "D0",
            "E",
        ]
    ]
    return df


def collect(M_comp, comp_list):
    df = pd.concat(comp_list)
    df["x"] = (df["w_mass"] / M_but) / (df["w_mass"] / M_but + (100 - df["w_mass"]) / M_comp) * 100
    return df


# OCM
comp_list = []
for i in range(9):
    file = [f"{PATH}\OCM{i}\{f}" for f in os.listdir(f"{PATH}\OCM{i}") if ".hdf5" in f][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))

ocm = collect(M_ocm, comp_list)

# PETA
comp_list = []
for i in range(5):
    file = [f"{PATH}\PETA{i}\{f}" for f in os.listdir(f"{PATH}\PETA{i}") if ".hdf5" in f][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))
peta = collect(M_peta, comp_list)

# Butanol
butanol = to_df(Experiment.load_hdf5(f"{PATH}\BUTANOL\data_BUT_up.hdf5"))
butanol["x"] = 100
butanol["w_vol"] = 100

# DMAG
comp_list = []
for i in range(5):
    file = [f"{PATH}\DMAG{i}\{f}" for f in os.listdir(f"{PATH}\DMAG{i}") if ".hdf5" in f][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))
dmag = collect(M_dmag, comp_list)

In [8]:
# All data
all_df = pd.concat([ocm, peta, butanol, dmag]).reset_index(drop=True)
DF = all_df.groupby(by=["compound", "x", "Temperature"]).mean().reset_index()
DF.to_csv('Datas/Viscosity_all_data.csv')
DF

,compound,x,Temperature,rho,w_mass,w_vol,time,Viscosity,Kin_viscosity,D,D0,E
0,butanol,100.000000,13.0,0.82668,100.00,100.000000,127.760000,3.700000,4.475734,77337.837838,2.942474e+08,19660.651841
1,butanol,100.000000,13.1,0.82668,100.00,100.000000,87.724138,3.691379,4.465306,77545.607003,2.942474e+08,19660.651841
2,butanol,100.000000,13.2,0.82668,100.00,100.000000,101.250000,3.677500,4.448517,77868.040976,2.942474e+08,19660.651841
3,butanol,100.000000,13.3,0.82668,100.00,100.000000,10.750000,3.640000,4.403155,78695.054945,2.942474e+08,19660.651841
4,butanol,100.000000,13.4,0.82668,100.00,100.000000,199.500000,3.700000,4.475734,77445.945946,2.942474e+08,19660.651841
...,...,...,...,...,...,...,...,...,...,...,...,...
4840,peta,49.775459,40.1,1.11000,19.76,26.532153,10837.730769,21.326923,19.213444,14689.969173,8.236533e+11,46404.639425
4841,peta,49.775459,40.2,1.11000,19.76,26.532153,10896.565217,21.153623,19.057318,14814.297577,8.236533e+11,46404.639425
4842,peta,49.775459,40.3,1.11000,19.76,26.532153,11027.415584,21.193506,19.093249,14789.927545,8.236533e+11,46404.639425
4843,peta,49.775459,40.4,1.11000,19.76,26.532153,11416.136929,21.000000,18.918919,14930.952381,8.236533e+11,46404.639425


# plots

In [9]:
DF = pd.read_csv('Datas/Viscosity_all_data.csv',index_col=0)

In [10]:
# Simple data
datas = []
for _, group in DF.groupby(by=['compound', 'x']):
    for i0 in [15, 20, 25, 30, 40]:
        window_df = group[(i0 - 1 <= group['Temperature']) & (group['Temperature'] <= i0 + 1)]
        window_note = window_df[
            ['x', 'Temperature', 'rho', 'w_mass', 'w_vol', 'D', 'Viscosity', 'E', 'D0']
        ].mean()
        window_note['compound'] = group['compound'].iloc[0]
        window_note['Temperature'] = i0
        if any(window_note.isna()):
            continue
        datas.append(window_note)

DF2 = pd.concat(datas, axis=1).T
DF2.to_csv('Datas/Viscosity.csv')
DF2

,x,Temperature,rho,w_mass,w_vol,D,Viscosity,E,D0,compound
0,100.0,15,0.82668,100.0,100.0,80056.837868,3.600238,19660.651841,294247430.086774,butanol
1,100.0,20,0.82668,100.0,100.0,90518.909212,3.241333,19660.651841,294247430.086774,butanol
2,100.0,25,0.82668,100.0,100.0,105751.209477,2.820228,19660.651841,294247430.086774,butanol
3,100.0,30,0.82668,100.0,100.0,120714.749126,2.511667,19660.651841,294247430.086774,butanol
4,100.0,40,0.82668,100.0,100.0,153901.190065,2.035,19660.651841,294247430.086774,butanol
...,...,...,...,...,...,...,...,...,...,...
91,49.775459,15,1.11,19.76,26.532153,3040.431432,95.076418,46404.639425,823653316011.137451,peta
92,49.775459,20,1.11,19.76,26.532153,4477.934233,65.599728,46404.639425,823653316011.137329,peta
93,49.775459,25,1.11,19.76,26.532153,6219.358653,47.991778,46404.639425,823653316011.137573,peta
94,49.775459,30,1.11,19.76,26.532153,8408.512228,36.102178,46404.639425,823653316011.137573,peta


In [11]:
res = {}
for compound in DF["compound"].unique():
    df_comp = DF[DF["compound"] == compound]
    xs = df_comp["x"].unique()
    rhos = np.array([df_comp[df_comp["x"] == x]["rho"].mean() for x in xs]) * 1000
    print(f"{compound: >10}:", *xs.astype(int))
    print("*" * 10, *rhos.astype(int))
    res[compound] = list(zip(xs.astype(int), rhos.astype(int)))

   butanol: 100
********** 826
      dmag: 0 11 22 35 39
********** 1069 1069 1050 1029 1019
       ocm: 0 12 22 31 38 44 49 54 58
********** 1720 1580 1569 1550 1539 1530 1510 1489 1490
      peta: 0 19 30 41 49
********** 1200 1180 1159 1140 1110
